In [11]:
from polarization import run_polarize_pipeline
from util import btreeToList
from infer_util import btreeToLength, listToString
#from infer_util import listToString
updateWordCategory = {"NN", "JJ", "VBZ", "RB"}
relationDictionary = ["<", ">", "A", "U", "|", "#"]
replaced_words = {
    "out of": "out-of",
    "none of the": "none-of-the",
    "all of the": "all-of-the",
    "some of the": "some-of-the",
    "most of the": "most-of-the",
    "many of the": "many-of-the",
    "several of the": "several-of-the",
    "some but not all": "some-but-not-all"
}

2020-12-05 03:27:53 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | gum       |
| pos       | gum       |
| lemma     | gum       |
| depparse  | gum       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-12-05 03:27:53 INFO: Use device: gpu
2020-12-05 03:27:53 INFO: Loading: tokenize
2020-12-05 03:27:56 INFO: Loading: pos
2020-12-05 03:27:57 INFO: Loading: lemma
2020-12-05 03:27:57 INFO: Loading: depparse
2020-12-05 03:27:58 INFO: Loading: sentiment
2020-12-05 03:27:59 INFO: Loading: ner
2020-12-05 03:28:00 INFO: Done loading processors!


In [12]:
sentence_set = ["Every dog that hates a monkey is a heavy cat", "Every dog runs fast"]

In [13]:
result = run_polarize_pipeline(sentence_set)[0]

100%|██████████| 2/2 [00:00<00:00,  3.95it/s]Every dog that hates a monkey is a heavy cat



In [14]:
for parse_out in result:
    print(parse_out[2])

[nsubj↑ [det↓ [DT every↑] [NN dog↓]] [advmod↑ [RB fast↑] [VBZ runs↑]]]


In [15]:
class Sent_Info:
    def __init__(self, content = ""):
        self.content = content
        self.markForm = None
        self.treeForm = None
        self.categoryForm = None
        #mark_sent()
    def mark_sent(self):
        if (self.content != ""):
            temp = run_polarize_pipeline([self.content])[0][0]
            self.markForm = temp[0]
            set_arg = []
            #self.treeForm = string_form_list(temp[2].split(" "), set_arg, 0,0)
            self.treeForm = temp[4]
            print(temp)
            #print(set_arg)
            self.categoryForm = temp[3]
        else:
            print("Should given some string")            


In [16]:
def prepare_info(info_set, sentences):
    for sent in sentences:
        info = Sent_Info(sent)
        info.mark_sent()
        info_set.add(info)


In [17]:
info_set = set()
prepare_info(info_set, sentence_set)

100%|██████████| 1/1 [00:00<00:00, 15.19it/s]('every↑ dog↓ runs↑ fast↑', 'Every dog runs fast', '[nsubj↑ [det↓ [DT every↑] [NN dog↓]] [advmod↑ [RB fast↑] [VBZ runs↑]]]', [(1, 'DT'), (2, 'NN'), (4, 'RB'), (3, 'VBZ')], <binarization.BinaryDependencyTree object at 0x00000210604CA9C8>)



In [18]:
'''def string_form_list(str_list, output, current_layer, target_layer):
    if(str_list == []):
        return output
    print("First element is" + str_list[0] + "With current layer " + str(current_layer) + "and Target layer " + str(target_layer))
    print(output)
    if(str_list[0][0] == '['):
        if(current_layer==target_layer):
            output.append(string_form_list(str_list[1:], [str_list[0][1:]], current_layer+1, target_layer+1))
            print("second line")
            print("second line input is:")
            print(str_list[1:])
            string_form_list(str_list[1:], output, current_layer+1, target_layer)
        else:
            string_form_list(str_list[1:], output, current_layer+1, target_layer)
    elif (str_list[0][len(str_list[0])-1] == ']'):
        
        if(current_layer == target_layer):
            output.append(str_list[0][:(len(str_list[0])-1)])
            print("layer_output is")
            print(output)
            print("\n")
            return output
        else:
            string_form_list(str_list[1:], output, current_layer-1, target_layer)

    elif(current_layer==target_layer):
        output.append(str_list[0])
        string_form_list(str_list[1:], output, current_layer, target_layer)
    else:
        string_form_list(str_list[1:], output, current_layer, target_layer)'''


'def string_form_list(str_list, output, current_layer, target_layer):\n    if(str_list == []):\n        return output\n    print("First element is" + str_list[0] + "With current layer " + str(current_layer) + "and Target layer " + str(target_layer))\n    print(output)\n    if(str_list[0][0] == \'[\'):\n        if(current_layer==target_layer):\n            output.append(string_form_list(str_list[1:], [str_list[0][1:]], current_layer+1, target_layer+1))\n            print("second line")\n            print("second line input is:")\n            print(str_list[1:])\n            string_form_list(str_list[1:], output, current_layer+1, target_layer)\n        else:\n            string_form_list(str_list[1:], output, current_layer+1, target_layer)\n    elif (str_list[0][len(str_list[0])-1] == \']\'):\n        \n        if(current_layer == target_layer):\n            output.append(str_list[0][:(len(str_list[0])-1)])\n            print("layer_output is")\n            print(output)\n            pri

In [19]:
list1 = [1]
list2 = [2]
list1.append(list2)
print(list1)

[1, [2]]


In [33]:
class Pattern:
    def __init__(self, left, right, relation = 0):
        self.left = left
        self.right = right
        self.relation = relation
    def prettyPrint(self):
        leftSide, _, _, _ = btreeToList(self.left, btreeToLength(self.left), replaced_words)
        rightSide, _, _, _ = btreeToList(self.right, btreeToLength(self.right), replaced_words)
        relationMark = relationDictionary[self.relation]
        #print(listToString(leftSide))
        print("This pattern is: " + listToString(leftSide) +" " + relationMark + " " + listToString(rightSide))
        

In [34]:
class Sentence_Base:
    def __init__(self):
        self.pInfos = {}
        self.hInfo = None

In [35]:
class Knowledge_Base:
    def __init__(self):
        self.patterns = set()
        self.wordLists = set()
        self.functions = {
           # "updateWordList"
        }
    def updatewordList(self, sentInfo):
        tree = sentInfo.treeForm
        #print(tree)
        traverse_down_update(tree, self.wordLists, self.checkWordUpdate)
    def extractPattern(self, sentInfo):
        tree = sentInfo.treeForm
        traverse_down_update(tree, self.patterns, self.extract_sub)
    def checkWordUpdate(self, tree, baseList):
        if tree.npos in updateWordCategory:
            baseList.add(tree.val)
    def extract_sub(self, tree, patterns):
        if tree.val == "det" and tree.left.val == "every":
            self.patterns.add(Pattern(tree.right, tree.parent.right))

In [36]:
def traverse_down_update(tree, baseList, func):
    if(tree == "N"):
        return
    else:
        func(tree, baseList)
        traverse_down_update(tree.left, baseList, func)
        traverse_down_update(tree.right, baseList, func)

In [37]:
k_base = Knowledge_Base()
s_base = Sentence_Base()
for premise in info_set:
    k_base.updatewordList(premise)
    k_base.extractPattern(premise)

In [38]:
for pattern in k_base.patterns:
    pattern.prettyPrint() 

This pattern is: [NN dog↓1] < [advmod↑5 [RB fast↑3] [VBZ runs↑4]]
This pattern is: [acl:relcl↓9 [nsubj↓7 [WDT that↓1] [obj=6 [det=4 [DT a=2] [NN monkey=3]] [VBZ hates=5]]] [NN dog↓8]] < [cop↑17 [VBZ is↑11] [det↑16 [DT a↑12] [amod↑15 [JJ heavy↑13] [NN cat↑14]]]]
